In [2]:
import torch 
import torch.nn as nn  
import numpy as np
from tqdm.auto import tqdm

import cv2
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast
from torch.cuda import amp
from torch.optim import lr_scheduler
import time
import gc
import copy
from collections import defaultdict
import torch.nn.functional as F
import matplotlib.pyplot as plt
import albumentations as A
from pathlib import Path
from PIL import Image
import segmentation_models_pytorch as smp
import time

from segmentation.scr.utils.metrics import dice_coef

import torch
from torch.utils.data import Dataset, DataLoader




#from segmentation.config import Configs as CFG
from segmentation.models.unet import unet
from segmentation.scr.utils import losses, transforms, rle_coding
from segmentation.scr.utils.utils import set_seed, save_model
from segmentation.scr.tilling_dataset import Tilling_Dataset
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

#from segmentation.scr
pd.options.mode.chained_assignment = None

c:\Users\123\Desktop\human vasculature\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\123\AppData\Local\Temp\ipykernel_24640\525301475.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
class CFG:

    # configs for tilling dataset kidney 1
    path_img_kidney1 = "data\\train\\kidney_1_dense\\images"
    path_lb_kidney1 = "data\\train\\kidney_1_dense\\labels"
    path_df_kidney_1_till = "data\\kidney_1_tilling.csv"
    tile_size = (256, 256)
    overlap_pct = 0
    cache_dir = "data"
    mean_till1 = (0.2425, 0.2425, 0.2425)
    std_till1 = (0.1766, 0.1766, 0.1766)

    # configs for tilling dataset kidney 3
    path_img_kidney3 = "data\\train\\kidney_3_sparse\\images"
    path_lb_kidney3 = "data\\train\\kidney_3_dense\\labels"
    path_df_kidney_3_till = "data\\kidney_3_tilling.csv"
    tile_size = (256, 256)
    overlap_pct = 0
    cache_dir = "data"

    # configs for transforms
    p_rot = 0.3
    p_aug = 0.3
    # cofigs for train / eval model
    random_seed = 0
    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_batch_size = 2
    n_accumulate = max(1, 8 // train_batch_size)
    valid_batch_size = train_batch_size * 2
    clip_norm=5
    epochs = 28
    lr = 3e-4
    dice_th = 0.5
    
    path_to_save_state_model = "weight"
    path_weight_model = "weight"
    
    
    

In [4]:
train_transform  = transforms.get_transform(transform_type='strong')
val_transform = transforms.get_transform(transform_type='val')

In [5]:
train_dataset = Tilling_Dataset(
    name_data='kidney_1_tilling',
    path_to_df=CFG.path_df_kidney_1_till,
    use_random_sub=True,
    empty_tile_pct=0,
    sample_limit=64,
    random_seed=42
)


Dataset contains 20744 empty and 33952 non-empty tiles.
Sample 0 empty and 64 non-empty tiles.


In [6]:
val_dataset = Tilling_Dataset(
    name_data='kidney_3_tilling',
    path_to_df=CFG.path_df_kidney_3_till,
    use_random_sub=True,
    empty_tile_pct=10, 
    sample_limit=65,
    random_seed=CFG.random_seed)

Dataset contains 14595 empty and 6447 non-empty tiles.
Sample 6 empty and 58 non-empty tiles.


In [7]:
val_dataset = Tilling_Dataset(
    name_data='kidney_3_tilling',
    path_to_df=CFG.path_df_kidney_3_till,
    use_random_sub=True,
    empty_tile_pct=10,
    sample_limit=32,
    random_seed=CFG.random_seed)

Dataset contains 14595 empty and 6447 non-empty tiles.
Sample 3 empty and 28 non-empty tiles.


In [8]:
set_seed(CFG.random_seed)

train_loader = DataLoader(train_dataset, batch_size=CFG.train_batch_size, num_workers=2, shuffle=True, pin_memory=True)
val_loader = DataLoader(train_dataset, batch_size=CFG.valid_batch_size, num_workers=2, shuffle=False, pin_memory=True)

In [9]:
model = unet.UNet(n_channels=3, n_classes=1, bilinear=False).to(CFG.device)
num_epoch = 4
loss_fn = losses.BCE_DICE(mode="BCE")
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
sheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max',factor=0.9)
device = CFG.device

In [10]:
def train_one_loop(model, optimizer, loss_func, train_loader, device=CFG.device, grad_clip = None):
    model.train()
    running_loss = 0.0
    epoch_loss = 0.0

    dataset_size = 0
    optimizer.zero_grad()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc='Train ')
    for step, batch in pbar:
        images, masks,_, _ = batch
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        
        batch_size = images.shape[0]
        dataset_size += batch_size
    
        y_pred = model(images)
            
        loss   = loss_func(y_pred, masks)
        loss   = loss / CFG.n_accumulate
        loss.backward() #loss.backward()  # backward-pass

        running_loss += loss.item() * batch_size
        dataset_size += batch_size
        del images
        del masks
        del y_pred
        
        if (step + 1) % CFG.n_accumulate == 0 or (step + 1 == len(train_loader)):
            if grad_clip:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=CFG.clip_norm)
            optimizer.step()  # update weights
            optimizer.zero_grad()



        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix( epoch=f'{step + 1}',
                          train_loss=f'{running_loss / dataset_size:0.4f}',
                          #train_dice = f'{train_dice:0.4f}',
                          #train_jaccard = f'{train_jaccard:0.4f}',
                          lr=f'{current_lr:0.5f}',
                          gpu_mem=f'{mem:0.2f} GB')
    epoch_loss = running_loss / dataset_size

    gc.collect()
    torch.cuda.empty_cache()
    
    return epoch_loss
        

In [11]:
train_one_loop(model=model,optimizer=optimizer, train_loader=train_loader,
               loss_func=loss_fn, device=CFG.device)

Train : 100%|██████████| 32/32 [00:10<00:00,  3.06it/s, epoch=32, gpu_mem=1.65 GB, lr=0.00010, train_loss=0.0612]


0.0612151570385322

In [15]:
def valid_one_epoch(model, dataloader, loss_func,device=CFG.device):
 #(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0
    val_scores = []
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for _, batch in pbar:
        images, masks,_, _ = batch 
        images  = images.to(device, dtype=torch.float)
        masks   = masks.to(device, dtype=torch.float)

        batch_size = images.size(0)
        with torch.no_grad():
            
            y_pred  = model(images)
            loss    = loss_func(y_pred, masks)

        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        

        
        val_dice = dice_coef(y_pred=y_pred, y_true=masks).cpu().detach().numpy()
        val_scores.append([val_dice])
        del(images)
        del(masks)
        del(y_pred)
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(valid_loss=f'{running_loss / dataset_size:0.4f}',
                        gpu_memory=f'{mem:0.2f} GB')
    val_score  = np.mean(val_scores, axis=0)
    epoch_loss = running_loss / dataset_size
    
    torch.cuda.empty_cache()
    gc.collect()
    return epoch_loss, val_score[0]

In [16]:
valid_one_epoch(model=model, loss_func=loss_fn, dataloader=val_loader)

Valid : 100%|██████████| 16/16 [00:05<00:00,  3.14it/s, gpu_memory=1.08 GB, valid_loss=0.4299]


(0.4299070443958044, 0.026378512)

In [19]:
def train_model(model, 
                optimizer, 
                loss_func, 
                train_loader, 
                val_loader,
                num_epochs=1,
                grad_clip = None , 
                scheduler=None, 
                device =CFG.device,
                path_to_save=CFG.path_to_save_state_model,
                
):
    
    best_metric      = -np.inf
    loss_mas, metrics_mas,train_losses, val_losses = [], [], [],[]
    best_epoch = 0
    total_time = 0
    for epoch in range(num_epochs):
        start = time.time()
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)
        train_loss = train_one_loop(model=model, 
                                    optimizer=optimizer, 
                                    loss_func=loss_func,
                                    train_loader=train_loader,
                                    grad_clip=grad_clip,
                                    device=device)
        val_loss, dice_metric = valid_one_epoch(model=model,
                                                loss_func=loss_fn,
                                                dataloader=val_loader,
                                                device=device
                                                )
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        metrics_mas.append(dice_metric)
        if scheduler:
            if scheduler.__class__.__name__ == 'ReduceLROnPlateau':
                scheduler.step(dice_metric)
            else:
                scheduler.step()
        if loss_func.__class__.__name__:
            loss_func.update_n()
            
        # deep copy the model
        print(f"Epoch #{epoch+1} train loss: {train_loss:.3f}")
        print(f"Epoch #{epoch+1} val loss: {val_loss:.3f}")
        print(f"Epoch #{epoch+1} dice_metric: {dice_metric}")
        loss_mas.append(losses)
        
        if dice_metric > best_metric:
            print(f"{c_}Valid metrics Improved ({best_metric} ---> {dice_metric})")
            best_metric    = dice_metric
            save_model(
                model=model,
                optimizer=optimizer,
                model_name=model.__class__.__name__ +"_best_model_at_" + str(epoch + 1),
                path=path_to_save,
                lr_scheduler=scheduler,
            )
        
        with open("train_results.txt", "w") as file_handler:
            file_handler.write("train_loss\n")
            for item in train_losses:
                file_handler.write("{}\t".format(item))

            file_handler.write("\nval_loss\n")
            for item in val_losses:
                file_handler.write("{}\t".format(item))

            file_handler.write("\ndice_metric\n")
            for item in metrics_mas:
                file_handler.write("{}\t".format(item))
        
        
        end = time.time()
        total_time += end - start
        print(f"{sr_}Took {((end - start) / 60):.3f} minutes for epoch {epoch + 1}")
        
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        total_time // 3600, (total_time % 3600) // 60, (total_time % 3600) % 60))

        
        
train_model(model=model, 
                optimizer=optimizer, 
                loss_func=loss_fn, 
                train_loader=train_loader, 
                val_loader=val_loader, 
                num_epochs=3, 
                scheduler=sheduler, 
                device =CFG.device,
                path_to_save=CFG.path_to_save_state_model)

Epoch 1/3
----------


Valid : 100%|██████████| 16/16 [00:05<00:00,  3.18it/s, gpu_memory=1.08 GB, valid_loss=0.2473]


Epoch #1 train loss: 0.032
Epoch #1 val loss: 0.247
Epoch #1 dice_metric: 0.018457788974046707
Valid loss Improved (-inf ---> 0.018457788974046707)
Took 0.237 minutes for epoch 1
Epoch 2/3
----------


Valid : 100%|██████████| 16/16 [00:04<00:00,  3.21it/s, gpu_memory=1.08 GB, valid_loss=0.2531]


Epoch #2 train loss: 0.031
Epoch #2 val loss: 0.253
Epoch #2 dice_metric: 0.024851622059941292
Valid loss Improved (0.018457788974046707 ---> 0.024851622059941292)
Took 0.234 minutes for epoch 2
Epoch 3/3
----------


Valid : 100%|██████████| 16/16 [00:04<00:00,  3.22it/s, gpu_memory=1.08 GB, valid_loss=0.2439]


Epoch #3 train loss: 0.031
Epoch #3 val loss: 0.244
Epoch #3 dice_metric: 0.03110937774181366
Valid loss Improved (0.024851622059941292 ---> 0.03110937774181366)
Took 0.233 minutes for epoch 3
Training complete in 0h 0m 42s


In [ ]:
loss_fn.n

4